In [15]:
import pandas as pd
import re
from rapidfuzz import process, fuzz
from collections import defaultdict

In [ ]:
# === 1. Caricamento dati ===
df_primo = pd.read_excel("dati_detenuti.xlsx")
df_suicidi = pd.read_excel("mortiCarcere.xlsx")

df_primo.fillna(0, inplace=True)
df_primo['istituto'] = df_primo['istituto'].str.replace(r'\s*-\s*', '', regex=True)

df_primo['capienza'] = df_primo['capienza'].astype(int)
df_primo['presenti'] = df_primo['presenti'].astype(int)

# === 2. Normalizzazione date ===
mesi = {
    "gen": "01", "feb": "02", "mar": "03", "apr": "04", "mag": "05", "giu": "06",
    "lug": "07", "ago": "08", "set": "09", "ott": "10", "nov": "11", "dic": "12"
}

def normalizza_data(data_str):
    if isinstance(data_str, str):
        abbrev = data_str[:3].lower()
        mese = mesi.get(abbrev)
        anno = "20" + data_str[-2:]
        return f"{mese}/{anno}" if mese else None
    return None

df_primo['Data_std'] = df_primo['data'].apply(normalizza_data)
df_suicidi['Data_std'] = pd.to_datetime(df_suicidi['Data'], errors='coerce').dt.strftime("%m/%Y")

# === 3. Normalizzazione nomi istituti ===
def normalizza_nome(nome):
    if pd.isna(nome):
        return ""
    nome = re.sub(r"[\(\)\.\-\"']", "", nome)
    nome = re.sub(r"\s+", " ", nome)
    return nome.upper().strip()

df_primo['istituto_norm'] = df_primo['istituto'].apply(normalizza_nome)
df_suicidi['Istituto_norm'] = df_suicidi['Istituto'].apply(normalizza_nome)

# === 4. Fuzzy matching iniziale ===
istituti_dataset = df_primo['istituto_norm'].dropna().unique().tolist()

def fuzzy_match(nome):
    if nome == "":
        return ""
    match, score, _ = process.extractOne(nome, istituti_dataset, scorer=fuzz.token_sort_ratio)
    return match if score >= 50 else nome

df_suicidi['Istituto_norm'] = df_suicidi['Istituto_norm'].apply(fuzzy_match)

# === 5. Conteggio suicidi e primo merge ===
suicidi_counts = df_suicidi.groupby(['Data_std', 'Istituto_norm']).size().reset_index(name='suicidi')

df_finale = df_primo.merge(
    suicidi_counts,
    how='left',
    left_on=['Data_std', 'istituto_norm'],
    right_on=['Data_std', 'Istituto_norm']
)

df_finale['suicidi'] = df_finale['suicidi'].fillna(0).astype(int)

# === 6. Colonne finali e esportazione iniziale ===
df_finale['istituto'] = df_primo['istituto']
df_finale['data'] = df_primo['data']
df_finale['capienza'] = df_primo['capienza']
df_finale['detenuti'] = df_primo['presenti']
df_finale['sovraffollamento'] = df_primo['sovraffollamento']

colonne_finali = ['istituto', 'capienza', 'detenuti', 'data', 'sovraffollamento', 'suicidi']
df_finale = df_finale[colonne_finali]
df_finale.to_excel("dataset_con_suicidi.xlsx", index=False)
print("✅ File esportato in 'dataset_con_suicidi.xlsx'")

# === 7. Verifica chiavi non matchate ===
df_suicidi['chiave_merge'] = df_suicidi['Data_std'] + " | " + df_suicidi['Istituto_norm']
df_primo['chiave_merge'] = df_primo['Data_std'] + " | " + df_primo['istituto_norm']

chiavi_primo = set(df_primo['chiave_merge'].unique())
chiavi_suicidi = df_suicidi['chiave_merge'].unique()

mancanti = [c for c in chiavi_suicidi if c not in chiavi_primo]
print(f"🔎 {len(mancanti)} righe non matchate nel merge:")
for m in mancanti:
    print(m)

istituti_non_matchati = {c.split(" | ")[1] for c in mancanti}
for nome in istituti_non_matchati:
    match, score, _ = process.extractOne(nome, df_primo['istituto_norm'].unique(), scorer=fuzz.token_sort_ratio)
    print(f"{nome:<40} → {match:<40} (score: {score})")

# === 8. Fuzzy matching condizionato per chiavi non trovate ===
df_primo_by_data = defaultdict(list)
for _, row in df_primo.iterrows():
    df_primo_by_data[row["Data_std"]].append(row["istituto_norm"])

righe_corrette = 0
soglia = 60

for i, row in df_suicidi.iterrows():
    chiave = f"{row['Data_std']} | {row['Istituto_norm']}"
    if chiave in chiavi_primo:
        continue
    candidati = df_primo_by_data.get(row['Data_std'], [])
    if candidati:
        match, score, _ = process.extractOne(row['Istituto_norm'], candidati, scorer=fuzz.token_sort_ratio)
        if score >= soglia:
            df_suicidi.at[i, 'Istituto_norm'] = match
            righe_corrette += 1

print(f"🔄 Righe corrette tramite fuzzy matching condizionato: {righe_corrette}")

# === 9. Ricalcolo merge finale ===
df_suicidi['chiave_merge'] = df_suicidi['Data_std'] + " | " + df_suicidi['Istituto_norm']
df_primo['chiave_merge'] = df_primo['Data_std'] + " | " + df_primo['istituto_norm']

suicidi_counts = df_suicidi.groupby(['Data_std', 'Istituto_norm']).size().reset_index(name='suicidi')

df_finale = df_primo.merge(
    suicidi_counts,
    how='left',
    left_on=['Data_std', 'istituto_norm'],
    right_on=['Data_std', 'Istituto_norm']
)

df_finale['suicidi'] = df_finale['suicidi'].fillna(0).astype(int)

# === 10. Colonne finali e esportazione finale ===
df_finale['istituto'] = df_primo['istituto']
df_finale['data'] = df_primo['data']
df_finale['capienza'] = df_primo['capienza']
df_finale['detenuti'] = df_primo['presenti']
df_finale['sovraffollamento'] = df_primo['sovraffollamento']

df_finale = df_finale[colonne_finali]
df_finale.to_excel("dataset_con_suicidi.xlsx", index=False)
print("✅ File aggiornato con merge migliorato: 'dataset_con_suicidi.xlsx'")

df_finale.dtypes

C:\Users\valde\AppData\Local\Temp\ipykernel_13048\3217792593.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_suicidi['Data_std'] = pd.to_datetime(df_suicidi['Data'], errors='coerce').dt.strftime("%m/%Y")


✅ File esportato in 'dataset_con_suicidi.xlsx'
🔎 30 righe non matchate nel merge:
04/2025 | CUNEO
04/2025 | NAPOLI P MANDATO SECONDIGLIANO
12/2024 | VITERBO NC
09/2024 | BENEVENTO
08/2024 | CAMERINO
01/2024 | ARIENZO
08/2023 | BERGAMO
07/2023 | PESCARA
07/2023 | RAGUSA
07/2023 | PARMA
07/2023 | LARINO
07/2023 | VICENZA
08/2022 | ARIENZO
12/2021 | CAMERINO
07/2021 | SANREMO
05/2021 | BERGAMO
07/2020 | CAMERINO
02/2020 | ARIENZO
09/2019 | NAPOLI G SALVIA POGGIOREALE
07/2019 | NAPOLI P MANDATO SECONDIGLIANO
07/2019 | FERRARA C SATTA
07/2019 | NAPOLI G SALVIA POGGIOREALE
06/2019 | NAPOLI G SALVIA POGGIOREALE
06/2019 | BOLOGNA R DAMATO
06/2019 | MILANO F DI CATALDO SAN VITTORE
05/2019 | BOLOGNA R DAMATO
05/2019 | CAGLIARI ESCALAS
03/2019 | MILANO F DI CATALDO SAN VITTORE
03/2019 | TERMINI IMERESE A BURRAFATO
02/2019 | CAGLIARI ESCALAS
TERMINI IMERESE A BURRAFATO              → TERMINI IMERESE A BURRAFATO              (score: 100.0)
BENEVENTO                                → BENEVENTO       

istituto             object
capienza              int64
detenuti              int64
data                 object
sovraffollamento    float64
suicidi               int64
dtype: object